In [6]:
import os
from pinecone import Pinecone,ServerlessSpec
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()

In [9]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 24.51it/s]


In [10]:
index=pc.Index(index_name)

In [11]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [ ]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",

]
)

100%|██████████| 1/1 [00:04<00:00,  4.15s/it]


In [13]:
retriever.invoke("What city did i visit first")

[Document(metadata={'score': 0.21249935}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.239368573}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.232818738}, page_content='In 2022, I visited New York')]

In [1]:
from langchain_core.runnables import RunnableMap, RunnableSequence
